In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
#feature_list = ['x','y','hoursin','hourcos','weekdaysin','weekdaycos','daysin','daycos','monthsin','monthcos','year', 'accuracy']
feature_list = ['x','y','hour','hoursin','hourcos','weekday','day','month','year', 'accuracy']

In [3]:
def prepare_data(df, n_cell_x, n_cell_y):
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_x'] = pos_x
    df['grid_y'] = pos_y
    
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['hoursin'] = np.sin((d_times.hour + d_times.minute/60) * 0.2617994) * fw[8] # 0.2617994 = 2pi/24
    df['hourcos'] = np.cos((d_times.hour + d_times.minute/60) * 0.2617994) * fw[8] # 0.2617994 = 2pi/24
    df['weekday'] = d_times.weekday * fw[3]
    #df['weekdaysin'] = np.sin(d_times.weekday * 0.8975979) * fw[3] #0.8975979 = 2pi/7
    #df['weekdaycos'] = np.cos(d_times.weekday * 0.8975979) * fw[3] #0.8975979 = 2pi/7
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    #df['daysin'] = np.sin(d_times.dayofyear * 0.0172142) * fw[4] #0.0172142 = 2pi/365
    #df['daycos'] = np.cos(d_times.dayofyear * 0.0172142) * fw[4] #0.0172142 = 2pi/365
    df['month'] = d_times.month * fw[5]
    #df['monthsin'] = np.sin(d_times.month * 0.5235987) * fw[5] #0.5235987 = 2pi/12
    #df['monthcos'] = np.cos(d_times.month * 0.5235987) * fw[5] #0.5235987 = 2pi/12
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    
    df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def run_prediction(df_train, df_test, n_cell_x, n_cell_y, th):
    total_result = pd.DataFrame()
    for x_grid in range(0, n_cell_x):
        start_time = time.time()
        for y_grid in range(0, n_cell_y):
            total_result = total_result.append(process_one_cell(df_train, df_test, x_grid, y_grid, th))
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_grid, flush = True)
    return total_result

In [5]:
def process_one_cell(df_train, df_test, grid_x, grid_y, th):
    start_time = time.time()
    
    df_cell_train = df_train[((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y))]# |
    '''
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y - 1))]
    '''
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    df_cell_test = df_test[(df_test.grid_x == grid_x)&(df_test.grid_y == grid_y)]
    row_ids = df_cell_test.row_id
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=35, weights='distance', 
                               metric='euclidean')
    clf.fit(df_cell_train[feature_list], df_cell_train.place_id)
    predictions = clf.predict_proba(df_cell_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_cell_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [6]:
def run_validation():
    # Run validation
    df_train = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    df_valiation = df_train[df_train.time > 786239 * 0.875]
    
    df_train = df_train[df_train.time <= 786239 * 0.875]
    
    n_cell_x = 10
    n_cell_y = 20 
    
    print('Preparing train data')
    df_train = prepare_data(df_train, n_cell_x, n_cell_y)
    df_valiation = prepare_data(df_valiation, n_cell_x, n_cell_y)
    df_valiation['p1'] = np.nan
    df_valiation['p2'] = np.nan
    df_valiation['p3'] = np.nan
    
    prediction_result = run_prediction(df_train, df_valiation, n_cell_x, n_cell_y, 5)
    
    # Calculate score
    prediction_result.sort_index(inplace=True)
    prediction_result['score'] = (prediction_result.p1 == df_valiation.place_id) * 1
    prediction_result['score'] += (prediction_result.p2 == df_valiation.place_id) * 0.5
    prediction_result['score'] += (prediction_result.p3 == df_valiation.place_id) * 0.33
    
    print(prediction_result.score.describe(), flush=True)

In [7]:
#     x    y     hour weekday  day   month year  accuracy
fw = [400, 1000, 4,   3.5,  1./22., 2,    9,    0.09, 1]
run_validation()

#     x    y     hour weekday  day   month year  accuracy
fw = [400, 1000, 4,   3.5,  1./22., 2,    9,    0.09, 5]
run_validation()

#     x    y     hour weekday  day   month year  accuracy
fw = [400, 1000, 4,   3.5,  1./22., 2,    9,    0.09, 10]
run_validation()

#     x    y     hour weekday  day   month year  accuracy
fw = [400, 1000, 4,   3.5,  1./22., 2,    9,    0.09, 15]
run_validation()

#     x    y     hour weekday  day   month year  accuracy
fw = [400, 1000, 4,   3.5,  1./22., 2,    9,    0.09, 20]
run_validation()

Preparing train data
Elapsed time overall: 79.57291173934937 seconds 0
Elapsed time overall: 66.65984344482422 seconds 1
Elapsed time overall: 65.7468740940094 seconds 2
Elapsed time overall: 62.66032075881958 seconds 3
Elapsed time overall: 70.98210334777832 seconds 4
Elapsed time overall: 80.45829892158508 seconds 5
Elapsed time overall: 82.04556155204773 seconds 6
Elapsed time overall: 87.21699857711792 seconds 7
Elapsed time overall: 87.25236368179321 seconds 8
Elapsed time overall: 76.22524237632751 seconds 9
count    4417684.000000
mean           0.528521
std            0.451912
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
Elapsed time overall: 76.99919319152832 seconds 0
Elapsed time overall: 82.8597137928009 seconds 1
Elapsed time overall: 89.16269969940186 seconds 2
Elapsed time overall: 87.1369571685791 seconds 3
Elapsed time overall: 88.7612898349762 se

Validation result:
Pure sin and cos on hour
40  0.526709
45  0.524087
48  0.522535
50  0.521572
55  0.519202

with hour:
1   0.528521
5   0.528367
10  0.527838
15  0.526738
20  0.525233


In [ ]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_validation_result.csv', index=False)

In [ ]:
# Run test
df_train = pd.read_csv('../../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 30
n_cell_y = 60 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_result.csv', index=False)